<a href="https://colab.research.google.com/github/harrisontg57/NLP_Spring_2021/blob/main/Homework2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


Tim Harrison ID: 002-24-5948

In [132]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

Problem 1)

In [145]:
import re
import nltk
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


#Made this multiline so it's visible
#I didn't want to play around with detecting filenames
play_names = ["a-midsummer-nights-dream","alls-well-that-ends-well",
              "antony-and-cleopatra","as-you-like-it","coriolanus","cymbeline",
              "hamlet","henry-iv-part-1","henry-iv-part-2","henry-v",
              "henry-vi-part-1","henry-vi-part-2","henry-vi-part-3","henry-viii",
              "julius-caesar","king-john","king-lear","loves-labors-lost","lucrece",
              "macbeth","measure-for-measure","much-ado-about-nothing","othello",
              "pericles","richard-ii","richard-iii","romeo-and-juliet",
              "shakespeares-sonnets","the-comedy-of-errors","the-merchant-of-venice",
              "the-merry-wives-of-windsor","the-phoenix-and-turtle",
              "the-taming-of-the-shrew","the-tempest","the-two-gentlemen-of-verona",
              "the-two-noble-kinsmen","the-winters-tale","timon-of-athens",
              "titus-andronicus","troilus-and-cressida","twelfth-night","venus-and-adonis"]
play_strings = []

for title in play_names:
  with open('/content/'+title+'_TXT_FolgerShakespeare.txt', 'r') as file:
      d = file.read().replace('\n', ' ')
      play_strings.append(d)

#print(len(play_strings))

Tfidf_vect = TfidfVectorizer()
vector_matrix = Tfidf_vect.fit_transform(play_strings)

cosine_similarity_matrix = cosine_similarity(vector_matrix)
#print(cosine_similarity_matrix)
pd.DataFrame(data=cosine_similarity_matrix,index=play_names,columns=play_names)

,a-midsummer-nights-dream,alls-well-that-ends-well,antony-and-cleopatra,as-you-like-it,coriolanus,cymbeline,hamlet,henry-iv-part-1,henry-iv-part-2,henry-v,henry-vi-part-1,henry-vi-part-2,henry-vi-part-3,henry-viii,julius-caesar,king-john,king-lear,loves-labors-lost,lucrece,macbeth,measure-for-measure,much-ado-about-nothing,othello,pericles,richard-ii,richard-iii,romeo-and-juliet,shakespeares-sonnets,the-comedy-of-errors,the-merchant-of-venice,the-merry-wives-of-windsor,the-phoenix-and-turtle,the-taming-of-the-shrew,the-tempest,the-two-gentlemen-of-verona,the-two-noble-kinsmen,the-winters-tale,timon-of-athens,titus-andronicus,troilus-and-cressida,twelfth-night,venus-and-adonis
a-midsummer-nights-dream,1.000000,0.631304,0.535508,0.558296,0.598400,0.620939,0.568969,0.680975,0.689609,0.700111,0.655302,0.675116,0.622498,0.698385,0.538396,0.684187,0.592891,0.601151,0.699611,0.529856,0.579437,0.537670,0.471106,0.603672,0.680581,0.656500,0.549571,0.719082,0.421336,0.629888,0.533295,0.469728,0.535359,0.565072,0.512597,0.622916,0.651692,0.512734,0.637770,0.594044,0.536979,0.703066
alls-well-that-ends-well,0.631304,1.000000,0.567778,0.587629,0.643325,0.663902,0.606938,0.706379,0.729037,0.730225,0.684510,0.699907,0.645807,0.759069,0.562737,0.713673,0.634234,0.629752,0.713955,0.551379,0.629946,0.572345,0.501913,0.644267,0.715313,0.696608,0.563123,0.728780,0.450742,0.660686,0.570906,0.441895,0.560272,0.587737,0.545630,0.623416,0.701888,0.550485,0.624404,0.628097,0.582105,0.701983
antony-and-cleopatra,0.535508,0.567778,1.000000,0.486620,0.546355,0.558805,0.503147,0.595370,0.608665,0.612212,0.575599,0.592515,0.546872,0.623884,0.615028,0.600436,0.529092,0.525879,0.612492,0.472408,0.526779,0.470679,0.416628,0.537365,0.602005,0.582697,0.477430,0.618813,0.375829,0.551414,0.471839,0.386087,0.464393,0.499975,0.450505,0.526559,0.583647,0.460943,0.538654,0.525819,0.478213,0.604072
as-you-like-it,0.558296,0.587629,0.486620,1.000000,0.556105,0.572645,0.516991,0.614050,0.628657,0.629584,0.587446,0.602011,0.552831,0.642673,0.493303,0.610615,0.542701,0.543458,0.611143,0.473716,0.555455,0.499179,0.433529,0.550705,0.607934,0.593105,0.491874,0.638843,0.390579,0.576793,0.499612,0.395358,0.494680,0.510511,0.476894,0.543498,0.605064,0.468956,0.545681,0.541361,0.503250,0.610461
coriolanus,0.598400,0.643325,0.546355,0.556105,1.000000,0.630912,0.568630,0.668016,0.691222,0.692496,0.640687,0.659505,0.607348,0.717713,0.589035,0.671507,0.596151,0.592583,0.676132,0.533779,0.596405,0.531945,0.467123,0.612143,0.667092,0.649914,0.524629,0.670556,0.420243,0.626140,0.538288,0.429383,0.526014,0.556876,0.499527,0.598136,0.662306,0.524768,0.619025,0.593249,0.537841,0.660049
cymbeline,0.620939,0.663902,0.558805,0.572645,0.630912,1.000000,0.587096,0.692094,0.704672,0.706223,0.666535,0.685729,0.633626,0.730438,0.555113,0.698157,0.613115,0.611779,0.716295,0.542769,0.610988,0.549312,0.485258,0.628526,0.698701,0.676805,0.553809,0.725914,0.435377,0.643264,0.550484,0.451633,0.549603,0.580665,0.532711,0.612334,0.689788,0.534156,0.627968,0.608218,0.557567,0.704438
hamlet,0.568969,0.606938,0.503147,0.516991,0.568630,0.587096,1.000000,0.639843,0.654995,0.664537,0.615214,0.637331,0.589685,0.675528,0.503279,0.649176,0.560390,0.564473,0.645069,0.498739,0.550090,0.500290,0.442483,0.567842,0.651515,0.626068,0.501497,0.653345,0.399734,0.590129,0.496809,0.421673,0.491971,0.527394,0.470978,0.551241,0.620768,0.484409,0.565920,0.555961,0.506235,0.631191
henry-iv-part-1,0.680975,0.706379,0.595370,0.614050,0.668016,0.692094,0.639843,1.000000,0.907617,0.810973,0.753808,0.774941,0.716731,0.790389,0.598959,0.787351,0.663213,0.667166,0.766979,0.594733,0.639954,0.604970,0.516880,0.668379,0.796019,0.749770,0.604132,0.790790,0.483369,0.700494,0.661567,0.513471,0.582150,0.632461,0.549814,0.654592,0.726265,0.576661,0.679174,0.662676,0.597659,0.756832
henry-iv-part-2,0.689609,0.729037,0.608665,0.628657,0.691222,0.704672,0.654995,0.907617,1.000000,0.834144,0.760103,0.789106,0.738444,0.818222,0.610075,0.801739,0

Problem 2)

In [140]:
import numpy as np

def top_n(m,n):
  li = []
  i = np.identity(len(play_names))
  df =  pd.DataFrame(data=m,index=play_names,columns=play_names)
  ident = pd.DataFrame(data=i,index=play_names,columns=play_names)
  no_ones = df.sub(ident)
  for k in range(n):
    top = np.argmax(no_ones)
    li.append((play_names[int(top/len(play_names))],play_names[top%len(play_names)]))
    no_ones.at[play_names[int(top/len(play_names))],play_names[top%len(play_names)]] = 0
    no_ones.at[play_names[top%len(play_names)],play_names[int(top/len(play_names))]] = 0
  return li
top_n(cosine_similarity_matrix,10)

[('lucrece', 'venus-and-adonis'),
 ('henry-iv-part-1', 'henry-iv-part-2'),
 ('lucrece', 'shakespeares-sonnets'),
 ('henry-vi-part-1', 'henry-vi-part-2'),
 ('richard-ii', 'richard-iii'),
 ('shakespeares-sonnets', 'venus-and-adonis'),
 ('henry-v', 'henry-viii'),
 ('henry-vi-part-2', 'henry-vi-part-3'),
 ('henry-vi-part-2', 'richard-ii'),
 ('henry-v', 'king-john')]

Problem 3)

In [141]:
import nltk
from nltk import bigrams, trigrams
from collections import Counter, defaultdict

model = defaultdict(lambda: defaultdict(lambda: 0))

plays = []

for title in play_names:
  with open('/content/'+title+'_TXT_FolgerShakespeare.txt', 'r') as file:
    lines = [line.rstrip() for line in file]
    #remove non letters except '
    lines = [re.sub(r'[^a-zA-Z0-9_\']',' ',line) for line in lines]
    #Remove Capitalized words (ie character names)
    lines = [re.sub(r'[A-Z][A-Z]+','',line) for line in lines]
    plays.extend(lines)
#print(plays)

model_t = defaultdict(lambda: defaultdict(lambda: 0))
for sentence in plays:
    #print(nltk.word_tokenize(sentence))
    for w1, w2, w3 in trigrams(nltk.word_tokenize(sentence), pad_right=True, pad_left=True):
        model_t[(w1, w2)][w3] += 1

for w1_w2 in model_t:
    total_count = float(sum(model_t[w1_w2].values()))
    for w3 in model_t[w1_w2]:
        model_t[w1_w2][w3] /= total_count

#dict(model_t["the","time"])

model_b = defaultdict(lambda: defaultdict(lambda: 0))
for sentence in plays:
    #print(nltk.word_tokenize(sentence))
    for w1, w2 in bigrams(nltk.word_tokenize(sentence), pad_right=True, pad_left=True):
        model_b[(w1)][w2] += 1

for w1 in model_b:
    total_count = float(sum(model_b[w1].values()))
    for w2 in model_b[w1]:
        model_b[w1][w2] /= total_count

#dict(model_b["dog"])

Problem 4)

In [144]:
import random

# starting words

def sentence_generator(model, start_terms, number_sentences):
  text = start_terms[:]
  new_sentences = []
  if model == "trigram":
    for x in range(number_sentences):
        sentence_finished = False
        while not sentence_finished:
          # select a random probability threshold  
          r = random.random()
          accumulator = .0
          for word in model_t[tuple(text[-2:])].keys():
              accumulator += model_t[tuple(text[-2:])][word]
              # select words that are above the probability threshold
              if accumulator >= r:
                  text.append(word)
                  break
          if text[-2:] == [None, None]:
              sentence_finished = True
        new_sentences.append(' '.join([t for t in text if t]))
        text = start_terms[:]

  elif model == "bigram":
    for x in range(number_sentences):
        sentence_finished = False
        while not sentence_finished:
          # select a random probability threshold  
          r = random.random()
          accumulator = .0
          for word in model_b[text[-1]].keys():
              accumulator += model_b[text[-1]][word]
              # select words that are above the probability threshold
              if accumulator >= r:
                  text.append(word)
                  break
          if text[-2:] == [None, None]:
              sentence_finished = True
        new_sentences.append(' '.join([t for t in text if t]))
        text = start_terms[:]
  return new_sentences

print(sentence_generator("bigram",["when"],10))
print(sentence_generator("trigram",["when", "we"],5))

['when she is good She was lengthened hours whose offense never till thy thoughts Long time Against the worm', 'when he reserved', "when they 'd say in advantage thee straight", 'when thou waited on you it is painted bow Had been too old tune them but a good fellow is Gualtier or seven times Go thy forlorn soldier ordered', 'when he how how now', 'when it up the way To some notorious bawd With thy return we know the worthy senators that speaks', 'when a worthier Peace be preserved souls must perforce compelled stain not Make answer for my lips Thou hast wronged and so Go you Hugh promised me good souls the rabble of Orleans and such a desperate hand Ask nothing to love', "when the creaking of stocks or so strictly tied by our French I would my own destruction and thy foes than the expecters of your father 's feast two or the cares yet", "when I say To let me 'midst my father to my chance Where 's to win a bawd no excuse All goes hard by th ' arrearages The Count Claudio Lysander keep i